In [1]:
import os;
import sys;
import numpy as np;
from sklearn.linear_model import LinearRegression;

/home/development/kevin/installations/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [2]:
def eng_read_sudha_format( fname):
    model = dict();
    f = open( fname, 'r');
    lcnt = 0;
    for line in f:
        if lcnt == 0:
            lcnt += 1;
            continue;
        tokens = line.strip().split( ' ');
        key = tokens[ 0];
        val = np.asarray( [ float( token) for token in tokens[ 1:]]);
        word, pos, sid = key.split( '@');
        if pos == 'noun':
            pos = 'n';
        if pos == 'adjective':
            pos = 'a';
        if pos == 'verb':
            pos = 'v';
        if pos == 'adverb':
            pos = 'r';
        mkey = sid + '_' + pos;
        model[ mkey] = val;
    return model;

In [3]:
def hin_read_sudha_format( fname):
    model = dict();
    f = open( fname, 'r');
    lcnt = 0;
    for line in f:
        if lcnt == 0:
            lcnt += 1;
            continue;
        tokens = line.strip().split( ' ');
        key = tokens[ 0];
        val = np.asarray( [ float( token) for token in tokens[ 1:]]);
        word, sid = key.split( '@');
        mkey = sid;
        model[ mkey] = val;
    return model;

In [4]:
fprefix = '/home/development/kevin/workspace/linking/data/';
fname = fprefix + 'english-sense-vector_sb.txt';
eng_m = eng_read_sudha_format( fname);

In [6]:
to_remove = [];
for key in eng_m:
    val = eng_m[ key];
    if np.linalg.norm( val) < 1e-6:
        to_remove.append( key);
for elem in to_remove:
    del eng_m[ elem];

In [7]:
print( type( eng_m[ '2430830_n']));
print( len( eng_m[ '2430830_n']));
print( np.shape( eng_m[ '2430830_n']));


<class 'numpy.ndarray'>
300
(300,)


In [8]:
fname = fprefix + 'hindi-sense-vector_sb.txt';
hin_m = hin_read_sudha_format( fname);

In [9]:
print( type( hin_m[ '16074']));
print( len( hin_m[ '16074']));
print( np.shape( hin_m[ '16074']));

<class 'numpy.ndarray'>
200
(200,)


In [10]:
hlist = dict();
elist = dict();
link_list = [];
fname = fprefix + 'links.csv';
f = open( fname, 'r');
mcnt = 0; # number of direct links which are mismatch in pos
tcnt = 0;
cnt_dict = dict();
for line in f:
    tokens = line.strip().split( ',');
    if tokens[ 1] == 'ADJECTIVE':
        p1 = 'a';
    if tokens[ 1] == 'NOUN':
        p1 = 'n';
    if tokens[ 1] == 'VERB':
        p1 = 'v';
    if tokens[ 1] == 'ADVERB':
        p1 == 'r';
    if tokens[ 3] == 'ADJECTIVE':
        p2 = 'a';
    if tokens[ 3] == 'NOUN':
        p2 = 'n';
    if tokens[ 3] == 'VERB':
        p2 = 'v';
    if tokens[ 3] == 'ADVERB':
        p2 == 'r';
    k1 = tokens[ 0];
    k2 = tokens[ 2] + '_' + p2;
    if p1 != p2:
        mcnt += 1;
    else:
        if p1 not in cnt_dict:
            cnt_dict[ p1] = 0;
        cnt_dict[ p1] += 1
        link_list.append( ( k1, k2, p1));
    tcnt += 1;

In [11]:
print( mcnt);
print( cnt_dict[ 'a']);
print( cnt_dict[ 'n']);
print( cnt_dict[ 'v']);
print( tcnt);

11
3602
11709
2003
17325


In [12]:
print( len( link_list));

17314


In [13]:
avail_list = [];
for tup in link_list:
    k1, k2, pos = tup;
    if k1 not in hin_m:
        continue;
    if k2 not in eng_m:
        continue;
    avail_list.append( tup);
print( len( avail_list));

8407


In [14]:
def get_cosine_similarity( v1, v2):
    num = 0.0;
    den1 = 0.0;
    den2 = 0.0;
    for i in range( len( v1)):
        num += ( v1[ i] * v2[ i]);
        den1 += ( v1[ i] * v1[ i]);
        den2 += ( v2[ i] * v2[ i]);
    return num / (np.sqrt( den1) * np.sqrt( den2));

In [15]:
def h2e( train_list, test_list, bias_p = False):
    evlist = [];
    hvlist = [];
    for tup in train_list:
        k1, k2, pos = tup;
        hvlist.append( hin_m[ k1]);
        evlist.append( eng_m[ k2]);
    h_data = np.array( hvlist, dtype = 'f4');
    e_data = np.array( evlist, dtype = 'f4');
    dim = len( e_data[ 0]);
    print( dim);
    dmodel_list = [];
    print( 'Training', flush = True);
    for i in range( dim):
        print( 'Dim = ', i, flush = True);
        X = h_data;
        Y = e_data[ :, i];
        cur_model = LinearRegression( fit_intercept = bias_p);
        cur_model.fit( X, Y);
        dmodel_list.append( cur_model);
    evlist = [];
    hvlist = [];
    for tup in test_list:
        k1, k2, pos = tup;
        hvlist.append( hin_m[ k1]);
        evlist.append( eng_m[ k2]);
    inp_data = np.array( hvlist, dtype = 'f4');
    
    ypred = [];
    print( 'Translating', flush = True);
    for i in range( dim):
        print( 'Dim = ', i, flush = True);
        X = inp_data;
        cur_model = dmodel_list[ i];
        Yp = cur_model.predict( X);
        ypred.append( Yp);
    trans_syn_data = ypred[ 0];
    for i in range( 1, dim):
        trans_syn_data = np.column_stack( ( trans_syn_data, ypred[ i]));
    print( np.shape( trans_syn_data));
    return trans_syn_data;
   

In [16]:
#trans_syn_data = h2e( avail_list[ : 841 * 9], avail_list[ 841 * 9:]);
#trans_syn_data = h2e( avail_list[ : 841 * 9], avail_list[ 841 * 9:], bias_p = True);
trans_syn_data = h2e( avail_list, avail_list);

300
Training
Dim =  0
Dim =  1
Dim =  2
Dim =  3
Dim =  4
Dim =  5
Dim =  6
Dim =  7
Dim =  8
Dim =  9
Dim =  10
Dim =  11
Dim =  12
Dim =  13
Dim =  14
Dim =  15
Dim =  16
Dim =  17
Dim =  18
Dim =  19
Dim =  20
Dim =  21
Dim =  22
Dim =  23
Dim =  24
Dim =  25
Dim =  26
Dim =  27
Dim =  28
Dim =  29
Dim =  30
Dim =  31
Dim =  32
Dim =  33
Dim =  34
Dim =  35
Dim =  36
Dim =  37
Dim =  38
Dim =  39
Dim =  40
Dim =  41
Dim =  42
Dim =  43
Dim =  44
Dim =  45
Dim =  46
Dim =  47
Dim =  48
Dim =  49
Dim =  50
Dim =  51
Dim =  52
Dim =  53
Dim =  54
Dim =  55
Dim =  56
Dim =  57
Dim =  58
Dim =  59
Dim =  60
Dim =  61
Dim =  62
Dim =  63
Dim =  64
Dim =  65
Dim =  66
Dim =  67
Dim =  68
Dim =  69
Dim =  70
Dim =  71
Dim =  72
Dim =  73
Dim =  74
Dim =  75
Dim =  76
Dim =  77
Dim =  78
Dim =  79
Dim =  80
Dim =  81
Dim =  82
Dim =  83
Dim =  84
Dim =  85
Dim =  86
Dim =  87
Dim =  88
Dim =  89
Dim =  90
Dim =  91
Dim =  92
Dim =  93
Dim =  94
Dim =  95
Dim =  96
Dim =  97
Dim =  98
Dim =  

In [25]:
import pickle;

print( 'Ranking', flush = True);

test_list = avail_list;
#test_list = avail_list[ 841 * 9:];


print( np.shape( trans_syn_data));
evlist = [];
hvlist = [];
ptr = 0;
k2id = dict();
for key in eng_m:
    evlist.append( eng_m[ key]);
    k2id[ key] = ptr;
    ptr += 1;
e_data = np.asarray( evlist, dtype = 'f4');
print( np.shape( e_data));

ntrans_syn_data = ( trans_syn_data.T / np.linalg.norm( trans_syn_data, axis = 1)).T;
ne_data = ( e_data.T / np.linalg.norm( e_data, axis = 1)).T;
print( np.shape( trans_syn_data));
print( np.shape( ntrans_syn_data));
print( np.shape( e_data));
print( np.shape( ne_data));

sim_mat = ntrans_syn_data.dot( ne_data.T);

Ranking
(8407, 300)
(48358, 300)
(8407, 300)
(8407, 300)
(48358, 300)
(48358, 300)


In [18]:
print( np.shape( sim_mat));

(8407, 48358)


In [19]:
r, c = np.shape( sim_mat);

In [20]:
trim_sim_mat = np.sort( sim_mat)[ :, ::-1]
trim_ind_mat = np.argsort( sim_mat)[ :, ::-1]

In [21]:
print( np.shape( trim_sim_mat));
print( np.shape( trim_ind_mat));

(8407, 48358)
(8407, 48358)


In [22]:
indx = [];
posl = [];
for i in range( r):
    true = test_list[ i][ 1];
    pos = test_list[ i][ 2];
    tlist = trim_ind_mat[ i].tolist();
    tind = tlist.index( k2id[ true]);
    indx.append( tind);
    posl.append( pos);

In [23]:
acc_01 = sum( [ elem < 1 for elem in indx]);
acc_03 = sum( [ elem < 3 for elem in indx]);
acc_05 = sum( [ elem < 5 for elem in indx]);
acc_08 = sum( [ elem < 8 for elem in indx]);
acc_10 = sum( [ elem < 10 for elem in indx]);
acc_20 = sum( [ elem < 20 for elem in indx]);
acc_50 = sum( [ elem < 50 for elem in indx]);
acc_100 = sum( [ elem < 100 for elem in indx]);

In [24]:
print( acc_01 / 8410);
print( acc_03 / 8410);
print( acc_05 / 8410);
print( acc_08 / 8410);
print( acc_10 / 8410);
print( acc_20 / 8410);
print( acc_50 / 8410);
print( acc_100 / 8410);
print( min( indx), max( indx), len( indx));

0.09892984542211652
0.19357907253269918
0.24007134363852556
0.28478002378121287
0.30535077288941737
0.36159334126040427
0.4334126040428062
0.49227110582639716
0 48238 8407


In [159]:
print( test_list[ : 10]);
print( trim_ind_mat[ : 10]);

[('33093', '14377617_n', 'n'), ('33140', '9862345_n', 'n'), ('33154', '2964634_n', 'n'), ('33269', '839778_n', 'n'), ('33273', '331102_n', 'n'), ('33508', '3956157_n', 'n'), ('33550', '13667643_n', 'n'), ('33595', '14941407_n', 'n'), ('33664', '11502102_n', 'n'), ('33684', '1067070_n', 'n')]
[[46331 18950 20428 ..., 43673 38788 36346]
 [48396 16127 16136 ..., 32260 32259     0]
 [48396 16127 16136 ..., 32260 32259     0]
 ..., 
 [48396 16127 16136 ..., 32260 32259     0]
 [48396 16127 16136 ..., 32260 32259     0]
 [19661 16982 21814 ..., 13461 12002 22947]]


In [148]:
print( np.sort( tp)[ :, ::-1]);
print( np.argsort( tp)[ :, ::-1]);

[[3 2 1]
 [3 2 1]
 [3 2 1]]
[[1 2 0]
 [1 0 2]
 [0 1 2]]


In [147]:
x = np.sort( tp)[ :,:: -1];
print( x) ;

[[3 2 1]
 [3 2 1]
 [3 2 1]]


In [19]:
zecnt = 0;
for key in eng_m:
    val = eng_m[ key];
    if np.linalg.norm( val) < 1e-6:
        zecnt += 1;
print( zecnt);

39


In [22]:
zhcnt = 0;
for key in hin_m:
    val = hin_m[ key];
    if np.linalg.norm( val) < 1e-15:
        zhcnt += 1;
print( zhcnt);

0
